# Threads
For getting started understanding threads in Python 3, I found this [Tutorials Point article](https://www.tutorialspoint.com/python3/python_multithreading.htm) to be really helpful. That said, here are my answers to the coding questions in Chapter 15 of CTCI.

## Thread vs. Process
> *15.1: What's the difference between a thread and a process?*

A process is an instance of a program on your computer (e.g. Unity or Microsoft Word). It has it's own memory allocation, it's own scheduled time in the CPU, and it's isolated from other processes. So if Unity wants to talk to another application, they have to formally set up communication (e.g. *pipes*, *files*, and *sockets*) because it cannot access that process' memory without permission. 

A thread, on the other hand, is part of a process. Separate threads still have their own local memory and call stack, but if they change data for the process, all of the sibling threads can see it. They also need to fit within the parent process' CPU time.

## Context Switches
> *15.2: How would you measure the time spent in a context switch?*

So what's a context switch? Context switches occur when a single processor moves between different jobs. Control is handed over to the kernel, and it takes the current state of the thread/process and puts it in the waiting state, then brings the next highest priority process into the running state. The registers/memory for one process get saved and moved aside and replaced with the new process' previous state. 

To measure the time between context switches, CTCI recommends passing some sort of token back and forth between two processes. You can then use it as a sort of game of ping pong, and use the time stamps between sending the token and receiving it on the other side as the length of a context switch.

One important thing to keep in mind here is that you can't force the kernel to go exactly from one process to another. Depending on the priority of other running processes, it may use the gap between P1 and P2 to run another process first. Therefore it's hard to get an exact number. The best approach here is to run it hundreds of times, then take the minimum time difference as the closest estimate for the length of the context switch.

## Dining Philosophers

> *15.3: In the famous dining philosophers problem, a bunch of philosophers are sitting around a circular table with one chopstick between each of them. A philosopher needs both chopsticks to eat, and always picks up the left chopstick before the right one. A deadlock could potentially occur if all the philosophers reached for their left chopstick at the same time. Using threads and locks, implement a simulation of the dining philosophers problem that prevents deadlocks.*

My solution to this problem involves putting a lock on the chopsticks, and checking if both the left and right chopsticks are available before picking them up. Once they're both available, the philosopher takes a bite and places both chopsticks back on the table so the next person can use them (unhygenic, but hey). When the meal is done (in my case, when they've taken 10 bites), the philosopher gets up to go think about desert.

In [3]:
import threading
import time
import random

print_lock = threading.Lock()
print_statements = []

def philosophers_eating_chinese(philosophers, entrees):
    chopstick_lock = threading.Lock()
    chopsticks = [True] * len(philosophers)
    philosopher_threads = []
    
    for i, name in enumerate(philosophers):
        philosopher = Philosopher(i, name, entrees[i % len(entrees)], chopsticks, chopstick_lock)
        philosopher_threads.append(philosopher)
    
    with print_lock:
        print_statements.append("Let's eat!")
        
    for hungry_philosopher in philosopher_threads:
        hungry_philosopher.start()
    
    for full_philosopher in philosopher_threads:
        full_philosopher.join()
        
    with print_lock:
        for statement in print_statements:
            print(statement)
        print("Dinner is finished!")

    
class Philosopher (threading.Thread):
    def __init__(self, pos, name, meal, chopsticks, lock):
        threading.Thread.__init__(self)
        self.pos = pos
        self.name = name
        self.meal = meal
        self.chopsticks = chopsticks
        self.lock = lock
        with print_lock:
            print_statements.append (self.name + " took a seat at the table")
            
    def run(self):
        finished_eating = False
        has_chopsticks = False
        bites_remaining = 10
        bite_generator = random.Random()
        while not finished_eating:
            with self.lock:
                left_chopstick = self.pos
                right_chopstick = (self.pos + 1) % len(self.chopsticks)
                if self.chopsticks[left_chopstick] and self.chopsticks[right_chopstick]:
                    has_chopsticks = True
                    self.chopsticks[left_chopstick] = False
                    self.chopsticks[right_chopstick] = False
            if has_chopsticks:
                time.sleep(bite_generator.uniform(0.01, 0.05)) #Take a bite
                bites_remaining -= 1
                with print_lock:
                    if bites_remaining:
                        print_statements.append(self.name + " has " + str(bites_remaining) + " bites left of " + self.meal)
                    else:
                        print_statements.append(self.name + " has finished eating the " + self.meal)
                if bites_remaining == 0:
                    finished_eating = True

                with self.lock:
                    left_chopstick = self.pos
                    right_chopstick = (self.pos + 1) % len(self.chopsticks)
                    self.chopsticks[left_chopstick] = True
                    self.chopsticks[right_chopstick] = True
                    has_chopsticks = False
        with print_lock:
            print_statements.append(self.name + " got up to go philosophize about desert.")


### Test for Dining Philosophers
For clarity in the output, I have added a global **`print_statements`** list with a **`print_lock`** guarding it. This ensures that everything gets printed in the correct order, though not at runtime. Even though print is supposedly threadsafe in Python 3, my attempts to use it have resulted in a lot of garbage output.

In [5]:
# Clear print_statments
with print_lock:
    del print_statements[:]
    
philosophers = ["Aristotle", "Plato", "Nietzsche", "Confucius", "Chomsky", "Socrates"]
meals = ["kung pao chicken", "General Tso's chicken", "fried rice", "beef and potatoes", "egg rolls", "mock duck"]

philosophers_eating_chinese(philosophers, meals)

Aristotle took a seat at the table
Plato took a seat at the table
Nietzsche took a seat at the table
Confucius took a seat at the table
Chomsky took a seat at the table
Socrates took a seat at the table
Let's eat!
Aristotle has 9 bites left of kung pao chicken
Confucius has 9 bites left of beef and potatoes
Aristotle has 8 bites left of kung pao chicken
Confucius has 8 bites left of beef and potatoes
Aristotle has 7 bites left of kung pao chicken
Confucius has 7 bites left of beef and potatoes
Aristotle has 6 bites left of kung pao chicken
Confucius has 6 bites left of beef and potatoes
Aristotle has 5 bites left of kung pao chicken
Confucius has 5 bites left of beef and potatoes
Aristotle has 4 bites left of kung pao chicken
Confucius has 4 bites left of beef and potatoes
Aristotle has 3 bites left of kung pao chicken
Confucius has 3 bites left of beef and potatoes
Aristotle has 2 bites left of kung pao chicken
Confucius has 2 bites left of beef and potatoes
Aristotle has 1 bites left

### CTCI Solution
So in my code, I have a single lock that controls access to all of the chopsticks. However, a better solution would be to have a lock for each chopstick, and pass the left and right chopstick locks to the threads instead of a larger chopstick lock. After all, in my implementation, only one person can touch the table at a time, which is a little silly in practice. If that's the case though, you could run into synchronicity problems. Say everyone picks up their left chopstick, sees there is no right chopstick available. If the system is *perfectly* synchronized, we could induce a weird sort of deadlock where philosophers pick up the left, check for the right, put it down, then start over again.

However, a particularly nifty solution can be found in how you identify the "left chopstick". In my implementation, the i'th person's left chopstick is always located in the i-th index of the chopstick array. Similarly, the right chopstick is found by `(self.pos + 1) % len(self.chopsticks)`. Therfore, the last person's right chopstick is actually stored at index 0 (because of the circular nature of the array..*cough cough*...table). However, if instead of always choosing the left chopstick, the philosophers always chose the lower indexed chopstick, the last person would reach for their right chopstick (at `i=0`) rather than their left (at `i=len(chopsticks) - 1`). This little tweak allows the second to last philosopher to start eating, thus preventing deadlock!